In [5]:
from Electromagnetics.FDTD import *
import numpy as np
import time

tic = time.clock()

n_dim = 2
r0, r1, dr = [None]*3
S = 0.9
n_t = None
save_every = 1000
source_at_right = True

##conductivity
sig_0 = 1.0e9
tau_0 = 1.0e8

r_c = None
r0_eps, r1_eps = None, None
r0_deps, r1_deps = None, None
r0_sig, r1_sig = None, None
r0_dsig, r1_dsig = None, None
a_eps, b_eps = None, None
omega = 1.0#2.0*np.pi
s_pml = None
d_pml = None
set_pml = True
is_t_var = False
set_sheet_src = True
set_sig_bulk = False
is_sig_t_var = True

set_g_ppwg = False
r0_sig_l, r1_sig_l = None, None
r0_sig_u, r1_sig_u = None, None
ev_od_sign = -1.0   ## +1 : even   -1 : odd

L_0 = 2.0*np.pi/omega
if n_dim==1:
    L_0 = 1.0/omega
print('L_0:', L_0)

omega_m, k_m = None, None
    
q, alpha_1, alpha_2 = getPlasmonDispTopAir(omega, sig_0, tau_0)
print('q:', q, 'alpha_1:', alpha_1, 'alpha_2', alpha_2, sep='\n')
q, alpha_1, alpha_2 = 10.0, 10.0, 10.0

def f_alpha_y(r, r_ref):
    a_1r, a_1i = np.real(alpha_1), np.imag(alpha_1)
    a_2r, a_2i = np.real(alpha_2), np.imag(alpha_2)
    y, y0 = r[1], r_ref[1]
    res = np.exp(-a_1r*np.abs(y-y0))*np.cos(a_1i*np.abs(y-y0))*(y>=y0) \
        -np.exp(-a_2r*np.abs(y-y0))*np.cos(a_2i*np.abs(y-y0))*(y<y0)
    return res

def f_alpha_y_tf(r, r_ref):
    a_1r, a_1i = np.real(alpha_1), np.imag(alpha_1)
    a_2r, a_2i = np.real(alpha_2), np.imag(alpha_2)
    y, y0 = r[1], r_ref[1]
    res = np.exp(-a_1r*np.abs(y-y0))*np.cos(a_1i*np.abs(y-y0))*(y>=y0) \
        -np.exp(-a_1r*np.abs(y-y0))*np.cos(a_1i*np.abs(y-y0))*(y<y0)
    return res


if n_dim==3:
    r0 = np.array([0.0, 0.0, 0.0])
    r1 = np.array([2.0, 2.0, 2.0])
    dr = np.array([0.02, 0.02, 0.02])
    S /= np.sqrt(3)
    n_t = 200
    
    r_c = (r0+r1)/2.0
    w_eps = (r1-r0)
    w_eps[1] /= 10.0
    w_eps[2] /= 10.0
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    a_eps, b_eps = 10.0, 1.0
    
    d_pml = np.array([0.2, 0.2, 0.2])
    s_pml = np.array([1.0+1.0j, 1.0+1.0j, 1.0+1.0j])
elif n_dim==2:
    set_wg_pec = False
    a_eps, b_eps = 1.0, 1.0

    w_wg_pec = L_0/0.9/np.sqrt(a_eps)
    r0 = np.array([0.0, 0.0])
    r1 = np.array([1.8*L_0, 0.53*L_0])
    if set_wg_pec:
        r1[1] = w_wg_pec
    dr = np.array([L_0/200, L_0/200])
    S /= np.sqrt(2)
    n_t = 80000

    r_c = (r0+r1)/2.0
    w_eps = (r1-r0)
    w_eps[1] = L_0/1.5/np.sqrt(a_eps)
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    if set_wg_pec:
        w_eps = (r1-r0)
        r0_eps, r1_eps = r0, r1

    d_source_deps = np.array([2.0*L_0, 0])
    w_deps = np.array([r1_eps[0] - 2*d_source_deps[0], 0.5*w_eps[1]])
    r0_deps = r0_eps + d_source_deps
    r1_deps = r0_deps + w_deps

    d_pml = np.array([0.1*L_0, 0.1*L_0])
    s_pml = np.array([1.0+6.0j/L_0, 1.0+6.0j/L_0])
    if set_wg_pec:
        d_pml[1] = 0.0
        s_pml[1] = 1.0

    if set_sig_bulk:
        ##bulk
        w_sig = (r1-r0)
        w_sig[1] /= 2.0
        r0_sig = np.copy(r0)
        r1_sig = r0_sig + w_sig
    else:
        ##thin film
        w_sig = (r1-r0)
        w_sig[1] /= 1000.0
        r0_sig = r_c - w_sig/2.0
        r1_sig = r_c + w_sig/2.0

    d_source_dsig = np.array([0.3*L_0, 0])
    w_dsig = np.array([r1_sig[0] - 2*d_source_dsig[0], w_sig[1]])
    r0_dsig = r0_sig + d_source_dsig
    r1_dsig = r0_dsig + w_dsig



elif n_dim==1:
    r0 = np.array([0.0])
    r1 = np.array([40.0*L_0])
    dr = np.array([L_0/100])
    S /= np.sqrt(1)
    n_t = 10000

    r_c = (r0+r1)/2.0
    a_eps, b_eps = 1.0, 1.0
    w_eps = (r1-r0)/2
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    
    r0_deps = r0_eps
    r1_deps = r1_eps

    d_pml = np.array([0.5*L_0])
    s_pml = np.array([1.0+5.0j/L_0])

    r0_sig = r0_eps
    r1_sig = r1_eps

    
dt = S*min(dr)
d_v = np.prod(dr)

fdtd = FDTDYEE(r0, r1, dr, dt, N_is_even=[True]*n_dim)

print('fdtd.r0:', fdtd.r0, 'fdtd.r1:', fdtd.r1)
print('fdtd.dr:', fdtd.dr, 'fdtd.dt:', fdtd.dt, 'fdtd.N:', fdtd.N)
print('d_pml:', d_pml, 's_pml:', s_pml)

d_g_ppwg = None
r0_dsig_l, r1_dsig_l = None, None
if set_g_ppwg and n_dim==2:
    _d_ = int(0.02*L_0/fdtd.dr[1])*fdtd.dr[1]
    d_g_ppwg = np.array([0, _d_])
    r0_sig_u = r0_sig + d_g_ppwg
    r1_sig_u = r1_sig + d_g_ppwg
    r0_sig_l = r0_sig - d_g_ppwg
    r1_sig_l = r1_sig - d_g_ppwg

    d_source_dsig = np.array([0.3*L_0, 0])
    w_dsig = np.array([r1_sig_l[0] - 2*d_source_dsig[0], w_sig[1]])
    r0_dsig_l = r0_sig_l + d_source_dsig
    r1_dsig_l = r0_dsig_l + w_dsig


fdtd.AllocateSideArr_list(["E", "D", "Je", "eps", "sig", "Jec"])
fdtd.AllocateFaceArr_list(["H", "Jm"])
fdtd.SetSpatialPointsSide()
fdtd.SetSpatialPointsFace()

d_x_ = np.zeros(n_dim)
d_x_[0] = (r1[0]-r0[0])
d_x_pml = np.zeros(n_dim)
d_x_pml[0] = d_pml[0]

## J
r_j = r_c - d_x_*0.45 + set_pml*d_x_pml
je_mag = 0.1/dr[0]
if source_at_right:
    r_j = r_c + d_x_*0.45 - set_pml*d_x_pml
f_je = fdtd.GetPointSourceFunc(r_j, mag=je_mag, src_dir='y', em_type='e')
if set_sheet_src:
    assert n_dim>=2
    if not set_g_ppwg:
        f_mag = None
        if set_sig_bulk:
            f_mag = lambda r: je_mag*f_alpha_y(r, r_j)
        else:
            f_mag = lambda r: je_mag*f_alpha_y_tf(r, r_j)
        f_je = fdtd.GetSheetSourceFunc(r_j, mag=f_mag, src_dir='y', norm_dir='x', em_type='e')
    else:
        f_mag_l = lambda r: je_mag*f_alpha_y_tf(r, r_j-d_g_ppwg)*ev_od_sign
        f_mag_u = lambda r: je_mag*f_alpha_y_tf(r, r_j+d_g_ppwg)
        
        f_je_u = fdtd.GetSheetSourceFunc(r_j+d_g_ppwg, mag=f_mag_u, src_dir='y', norm_dir='x', em_type='e')
        f_je_l = fdtd.GetSheetSourceFunc(r_j-d_g_ppwg, mag=f_mag_l, src_dir='y', norm_dir='x', em_type='e')
        f_je = [lambda r: f_je_l[i](r)+f_je_u[i](r) for i in range(3)]

        
Je_0 = fdtd.UpdateFuncSide_space(fdtd.Je, f_je, getCopy=True)
je_0_max = [np.max(np.abs(j)) for j in Je_0]
print('r_j:', r_j, 'r0:', fdtd.r0, 'r1:', fdtd.r1)
print('je_0_max:', je_0_max)

## M
r_m = r_c - d_x_*0.45 + set_pml*d_x_pml                         #### correct r_m for other fdtd doc...
jm_mag = 0.1/dr[0]
if source_at_right:
    r_m = r_c + d_x_*0.45 - set_pml*d_x_pml
    jm_mag *= -1.0
f_jm = fdtd.GetPointSourceFunc(r_m, mag=jm_mag, src_dir='z', em_type='m')
if set_sheet_src:
    assert n_dim>=2
    if not set_g_ppwg:
        f_mag = None
        if set_sig_bulk:
            f_mag = lambda r: jm_mag*f_alpha_y(r, r_m)
        else:
            f_mag = lambda r: jm_mag*f_alpha_y_tf(r, r_m)
        f_jm = fdtd.GetSheetSourceFunc(r_m, mag=f_mag, src_dir='z', norm_dir='x', em_type='m')
    else:
        f_mag_l = lambda r: jm_mag*f_alpha_y_tf(r, r_m-d_g_ppwg)*ev_od_sign
        f_mag_u = lambda r: jm_mag*f_alpha_y_tf(r, r_m+d_g_ppwg)

        f_jm_u = fdtd.GetSheetSourceFunc(r_m+d_g_ppwg, mag=f_mag_u, src_dir='z', norm_dir='x', em_type='m')
        f_jm_l = fdtd.GetSheetSourceFunc(r_m-d_g_ppwg, mag=f_mag_l, src_dir='z', norm_dir='x', em_type='m')
        f_jm = [lambda r: f_jm_l[i](r)+f_jm_u[i](r) for i in range(3)]
        
        
Jm_0 = fdtd.UpdateFuncFace_space(fdtd.Jm, f_jm, getCopy=True)
jm_0_max = [np.max(np.abs(m)) for m in Jm_0]
print('jm_0_max:', jm_0_max)


fj_t = lambda t: np.sin(omega*t) 

f_eps = fdtd.FunctionBox(r0_eps, r1_eps, a_eps, b_eps)
eps = fdtd.UpdateFuncSide_space(fdtd.eps, [f_eps]*3, getCopy=True)

deps = None
f_deps = None
deps_out = []
if is_t_var:
    fdtd.AllocateSideArr_list(["deps"])
    del_eps = 0.1
    omega_m = 0.2*omega
    k_m = omega_m*np.sqrt(a_eps)
    if n_dim==2:
        k_ = ppwg_disp(omega, d=w_eps[1], eps_r=a_eps, n=1)
        k_m = -0.2*np.real(k_)
        dw = ppwg_dw(k=k_, dk=k_m, d=w_eps[1], eps_r=a_eps, n=[1,2])
        omega_m = np.real(dw)
        print('cutoffs', [ppwg_cutoff(r1[1], a_eps, i) for i in range(n_mode)])
    print('del_eps:', del_eps, 'omega_m:', omega_m, 'k_m:', k_m)
    a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r[0])
    if n_dim==1:
        a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r)
    f_deps = fdtd.FunctionBox(r0_deps, r1_deps, a_deps, b=0.0)
    fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t=0.0)

print('is_t_var:', is_t_var)

##conductivity
if not set_g_ppwg:
    f_sig = fdtd.FunctionBox(r0_sig, r1_sig, sig_0, b=0.0)
    sig = fdtd.UpdateFuncSide_space(fdtd.sig, [f_sig]*3, getCopy=True)
else:
    f_sig_l = fdtd.FunctionBox(r0_sig_l, r1_sig_l, sig_0, b=0.0)
    f_sig_u = fdtd.FunctionBox(r0_sig_u, r1_sig_u, sig_0, b=0.0)
    f_sig = lambda r: f_sig_l(r) + f_sig_u(r)
    sig = fdtd.UpdateFuncSide_space(fdtd.sig, [f_sig]*3, getCopy=True)

dsig = None
f_dsig = None
dsig_out = []
if is_sig_t_var:
    fdtd.AllocateSideArr_list(["dsig"])
    del_sig = 0.1*sig_0
    omega_m = 0.1
    k_m = 2.8 #3.83  #for ppwg  w_m=0.1 k_m=-0.45 (exciting odd)
    print('del_sig:', del_sig, 'omega_m:', omega_m, 'k_m:', k_m)
    a_dsig = lambda r, t: del_sig*np.cos(omega_m*t - k_m*r[0])
    if n_dim==1:
        a_dsig = lambda r, t: del_sig*np.cos(omega_m*t - k_m*r)
    if not set_g_ppwg:
        f_dsig = fdtd.FunctionBox(r0_dsig, r1_dsig, a_dsig, b=0.0)
        fdtd.UpdateFuncSide_spacetime(fdtd.dsig, [f_dsig]*3, t=0.0)
    else:
        f_dsig = fdtd.FunctionBox(r0_dsig_l, r1_dsig_l, a_dsig, b=0.0)
        fdtd.UpdateFuncSide_spacetime(fdtd.dsig, [f_dsig]*3, t=0.0)
        

print('is_sig_t_var:', is_sig_t_var)

if set_pml:
    fdtd.AllocateSideArr_list(["dF_D", "F_D", "F_S", "F_Seps"])
    fdtd.AllocateFaceArr_list(["dF_H", "F_H"])     

set_eps_out = True
set_j_out = False
set_dsig_out = True

set_T_pt2d = True
set_R_pt2d = True
r_T_pt2d = None
r_R_pt2d = None
ind_T_, ind_R_ = None, None
E_T_out, E_R_out = [], []
if n_dim==2:
    r_R_1d = r_j[0] - (r_j[0]-d_pml[0])/2.0
    r_T_1d = r1[0] - r_R_1d
    if source_at_right:
        r_R_1d = r_j[0] + (r1[0]-d_pml[0]-r_j[0])/2.0
        r_T_1d = r1[0] - r_R_1d
    r_T_pt2d = np.array([r_T_1d, r_j[1]])
    r_R_pt2d = np.array([r_R_1d, r_j[1]])
    print('r0:', r0, 'r1:', r1)
    print('r_T_p2d:', r_T_pt2d, 'r_R_pt2d:', r_R_pt2d)
    i_xyz_min, r_xyz_min, d_xyz_min = fdtd.FindClosestSides(r_T_pt2d)
    ind_T_ =  i_xyz_min[0]
    i_xyz_min, r_xyz_min, d_xyz_min = fdtd.FindClosestSides(r_R_pt2d)
    ind_R_ =  i_xyz_min[0]



if n_dim==3:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'x', 'O_dir':'x', 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j,  [{'A':fdtd.Je, 'A_dir':'y', 'O_dir':'x', 'name':'J'}])
elif n_dim<=2:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'x', 'O_dir':None, 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.Je, 'A_dir':'y', 'O_dir':None, 'name':'J'}])
if is_t_var and set_eps_out:
    if n_dim==3:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':'x', 'name':'Eps'}])
    elif n_dim<=2:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':None, 'name':'Eps'}])
if is_sig_t_var and set_dsig_out:
    if n_dim==3:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.dsig, 'A_dir':'x', 'O_dir':'x', 'name':'Sig'}])
    elif n_dim<=2:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.dsig, 'A_dir':'x', 'O_dir':None, 'name':'Sig'}])


n_saved = 0
        
if not set_pml:
    for i in range(n_t):
        t = float(i*dt)
        ## Je
        fdtd.UpdateSeperableFunc_Time(fdtd.Je, Je_0, fj_t, t)
        ## eps*dE/dt = curl H - Je - Jec
        if not is_t_var:
            ##update E first? or J_c?
            fdtd.UpdateSide_dAdt_CurlB_C(fdtd.E, fdtd.H, C_list=[fdtd.Je, fdtd.Jec], a=fdtd.eps, b=None, c_list=[-np.ones(3), -np.ones(3)])
            
            ## tau*dJc/dt + Jc = sig_0*E
            if is_sig_t_var:
                dsig = fdtd.UpdateFuncSide_spacetime(fdtd.dsig, [f_dsig]*3, t)
                sig_p_dsig = [fdtd.sig[i]+fdtd.dsig[i] for i in range(3)]
                fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.E, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=sig_p_dsig, c_list=[-np.ones(3)])
            else:
                fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.E, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=fdtd.sig, c_list=[-np.ones(3)])
        else:
            fdtd.UpdateSide_dAdt_CurlB_C(fdtd.D, fdtd.H, C_list=[fdtd.Je, fdtd.Jec], a=None, b=None, c_list=[-np.ones(3), -np.ones(3)])
            deps = fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t)
            eps_p_deps = [fdtd.eps[i]+fdtd.deps[i] for i in range(3)]
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=eps_p_deps)
            
            ## tau*dJc/dt + Jc = sig_0*E
            if is_sig_t_var:
                dsig = fdtd.UpdateFuncSide_spacetime(fdtd.dsig, [f_dsig]*3, t)
                sig_p_dsig = [fdtd.sig[i]+fdtd.dsig[i] for i in range(3)]
                fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.E, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=sig_p_dsig, c_list=[-np.ones(3)])
            else:
                fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.E, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=fdtd.sig, c_list=[-np.ones(3)])

        
        ## PEC
        fdtd.ResetSideWalls(fdtd.E)
        t += dt/2
        ## Jm
        fdtd.UpdateSeperableFunc_Time(fdtd.Jm, Jm_0, fj_t, t)
        ## dH/dt = -curl E - Jm
        fdtd.UpdateFace_dAdt_CurlB_C(fdtd.H, fdtd.E, C_list=[fdtd.Jm], a=None, b=-np.ones(3), c_list=[-np.ones(3)])

        if i%save_every==0:
            fdtd.SaveOutputs()
            n_saved += 1
else:
    assert not is_t_var
    walls = fdtd.GetWallsAllDic(d_pml, s_pml)
    print('walls:' , walls)
    print('d_mpl:', d_pml, 's_pml:', s_pml)
    sx_arr_s, sy_arr_s, sz_arr_s = fdtd.GetUPMLFactor(walls, eps_or_s='s', side_or_face='side')
    sx_arr_f, sy_arr_f, sz_arr_f = fdtd.GetUPMLFactor(walls, eps_or_s='s', side_or_face='face')
            
    c_fd0 = np.array([-np.imag(sy_arr_s[0]), -np.imag(sz_arr_s[1]), -np.imag(sx_arr_s[2])])
    c_fd1 = [np.imag(sx_arr_s[0]), np.imag(sy_arr_s[1]), np.imag(sz_arr_s[2])]
    c_d = [-np.imag(sz_arr_s[0]), -np.imag(sx_arr_s[1]), -np.imag(sy_arr_s[2])]

    c_fh0 = [-np.imag(sy_arr_f[0]), -np.imag(sz_arr_f[1]), -np.imag(sx_arr_f[2])]
    c_fh1 = [np.imag(sx_arr_f[0]), np.imag(sy_arr_f[1]), np.imag(sz_arr_f[2])]
    c_h = [-np.imag(sz_arr_f[0]), -np.imag(sx_arr_f[1]), -np.imag(sy_arr_f[2])]

    #plt.imshow(np.imag(sz_arr_s[0]))
    #plt.show()
    
    for i in range(n_t):
        t = float(i*dt)
        ## Je
        fdtd.UpdateSeperableFunc_Time(fdtd.Je, Je_0, fj_t, t)
        ## curl H
        curl_H = fdtd.GetCurlFace(fdtd.H)
        ## dF_D/dt + s_fd*F_D = curl H - Je - Jec
        fdtd.Update_aA_bB(fdtd.dF_D, curl_H, C_list=[fdtd.F_D, fdtd.Je, fdtd.Jec], c_list=[c_fd0, -np.ones(3), -np.ones(3)])
        ## dD/dt + s_d*D = dF_D/dt + s_fd*F_D
        fdtd.Update_adAdt_bB(fdtd.D, fdtd.dF_D, C_list=[fdtd.D, fdtd.F_D], c_list=[c_d, c_fd1])
        ## F_S = s_y*F_Dx/Eps_xx   --->   d(Eps_xx*F_S)/dt = dF_D/dt + s_yi*F_D
        ## Eps_xx*F_S = F_Seps
        fdtd.Update_adAdt_bB(fdtd.F_Seps, fdtd.dF_D, C_list=[fdtd.F_D], c_list=[-c_fd0])
        fdtd.Update_aA_bB(fdtd.F_S, fdtd.F_Seps, a=fdtd.eps)
            
        fdtd.Update_adAdt_bB(fdtd.F_D, fdtd.dF_D)
        ## eps*E 
        if not is_t_var:
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=fdtd.eps)
        else:
            deps = fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t)
            eps_p_deps = [fdtd.eps[j]+fdtd.deps[j] for j in range(3)]
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=eps_p_deps)

        ## tau*dJc/dt + Jc = sig_0*F_S
        if is_sig_t_var:
            dsig = fdtd.UpdateFuncSide_spacetime(fdtd.dsig, [f_dsig]*3, t)
            sig_p_dsig = [fdtd.sig[i]+fdtd.dsig[i] for i in range(3)]
            fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.F_S, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=sig_p_dsig, c_list=[-np.ones(3)])
        else:
            fdtd.Update_adAdt_bB(fdtd.Jec, B=fdtd.F_S, C_list=[fdtd.Jec], a=tau_0*np.ones(3), b=fdtd.sig, c_list=[-np.ones(3)])

        
        ## PEC
        fdtd.ResetSideWalls(fdtd.E)
        fdtd.ResetSideWalls(fdtd.D)
        t += dt/2
        
        ## Jm
        fdtd.UpdateSeperableFunc_Time(fdtd.Jm, Jm_0, fj_t, t)
        ## curl E
        curl_E = fdtd.GetCurlSide(fdtd.E)
        ## dF_M/dt + s_fm*F_M = -curl E - Jm
        fdtd.Update_aA_bB(fdtd.dF_H, curl_E, C_list=[fdtd.F_H, fdtd.Jm], b=-np.ones(3), c_list=[c_fh0, -np.ones(3)])
        ## dH/dt + s_h*H = dF_H/dt + s_fd*F_H
        fdtd.Update_adAdt_bB(fdtd.H, fdtd.dF_H, C_list=[fdtd.H, fdtd.F_H], c_list=[c_h, c_fh1])
        fdtd.Update_adAdt_bB(fdtd.F_H, fdtd.dF_H)
    
        if i%save_every==0:
            fdtd.SaveOutputs()
            n_saved += 1
            
        if n_dim==2 and set_T_pt2d:
            E_T_out.append(fdtd.E[0][ind_T_])
            E_R_out.append(fdtd.E[0][ind_R_])
    
toc = time.clock()
print('processing time: {}:{}'.format(int((toc-tic)/60), int((toc-tic)%60)))

L_0: 6.283185307179586
q:
(1.06066017178-7.36569563736e-10j)
alpha_1:
(0.353553390593-2.20970869121e-09j)
alpha_2
(3.18198051534-1.59590072143e-08j)
fdtd.r0: [ 0.  0.] fdtd.r1: [ 11.30973355   3.33008821]
fdtd.dr: [ 0.03141593  0.03141593] fdtd.dt: 0.0199929732217 fdtd.N: [360 106]
d_pml: [ 0.62831853  0.62831853] s_pml: [ 1.+0.95492966j  1.+0.95492966j]
r_j: [ 10.11592834   1.66504411] r0: [ 0.  0.] r1: [ 11.30973355   3.33008821]
je_0_max: [0.0, 2.7203908761903617, 0.0]
jm_0_max: [0.0, 0.0, 2.7203908761903617]
is_t_var: False
del_sig: 100000000.0 omega_m: 0.1 k_m: 2.8
is_sig_t_var: True
r0: [ 0.  0.] r1: [ 11.30973355   3.33008821]
r_T_p2d: [ 0.91106187  1.66504411] r_R_pt2d: [ 10.39867168   1.66504411]
walls: {('x', 'p'): [(1+0.95492965855137202j), 0.62831853071795862], ('x', 'n'): [(1+0.95492965855137202j), 0.62831853071795862], ('y', 'n'): [(1+0.95492965855137202j), 0.62831853071795862], ('y', 'p'): [(1+0.95492965855137202j), 0.62831853071795862]}
d_mpl: [ 0.62831853  0.62831853] 

In [58]:
import matplotlib.pyplot as plt

r_E, E_out_list = fdtd.GetOutputs('E')
x_E, y_E, z_E = [None]*3
if n_dim==3:
    x_E, y_E, z_E = r_E
elif n_dim==2:
    x_E, y_E = r_E
elif n_dim==1:
    x_E = r_E[0]
e_max = [np.max(np.abs(e)) for e in E_out_list]
print('max(e_max):', max(e_max))
e_max = max(e_max)
    
x_J, y_J, z_J = [None]*3
j_max = None
J_out_list = None
if set_j_out:
    r_J, J_out_list = fdtd.GetOutputs('J')
    if n_dim==3:
        x_J, y_J, z_J = r_J
    elif n_dim==2:
        x_J, y_J = r_J
    elif n_dim==1:
        x_J = r_J[0]
    j_max = [np.max(np.abs(j)) for j in J_out_list]
    print('max(j_max):', max(j_max))
    j_max = max(j_max)



x_deps, y_deps, z_deps = [None]*3
deps_out_list = None
deps_max = None
if is_t_var and set_eps_out:
    r_deps, deps_out_list = fdtd.GetOutputs('Eps')
    if n_dim==3:
        x_deps, y_deps, z_deps = r_deps
    elif n_dim==2:
        x_deps, y_deps = r_deps
    elif n_dim==1:
        x_deps = r_deps[0]
    deps_max = [np.max(np.abs(de)) for de in deps_out_list]
    print('max(deps_max):', max(deps_max))
    deps_max = max(deps_max)

    
x_dsig, y_dsig, z_dsig = [None]*3
dsig_out_list = None
dsig_max = None
if is_sig_t_var and set_dsig_out:
    r_dsig, dsig_out_list = fdtd.GetOutputs('Sig')
    if n_dim==3:
        x_dsig, y_dsig, z_dsig = r_dsig
    elif n_dim==2:
        x_dsig, y_dsig = r_dsig
    elif n_dim==1:
        x_dsig = r_dsig[0]
    dsig_max = [np.max(np.abs(de)) for de in dsig_out_list]
    print('max(dsig_max):', max(dsig_max))
    dsig_max = max(dsig_max)

print(x_E.shape, E_out_list[0].shape)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

if n_dim==2:
    fig = plt.imshow(E_out_list[-1].T, origin='lower', vmin=-e_max, vmax=e_max)
    plt.show()

    if is_t_var and set_eps_out:
        fig = plt.imshow(deps_out_list[-1].T, origin='lower', vmin=-deps_max, vmax=deps_max)
        plt.show()
    if is_sig_t_var and set_dsig_out:
        fig = plt.imshow(dsig_out_list[-1].T, origin='lower', vmin=-dsig_max, vmax=dsig_max)
        plt.show()
        
    if set_T_pt2d:
        T_arr = np.trim_zeros(np.array(E_T_out))
        T_max = np.max(T_arr)
        plt.plot(T_arr)
        plt.show()
        T_fft = np.fft.fft(T_arr)
        T_fft = np.abs(T_fft)/len(T_fft)
        freq = np.fft.fftfreq(T_fft.shape[-1], dt)
        plt.plot(2.0*np.pi*freq/omega, 2.0*T_fft)
        if omega_m==None:
            omega_m = 0.1*omega
        plt.gca().set_xlim([1.0-3.0*omega_m, 1.0+3.0*omega_m])
        plt.gca().set_ylim([0, T_max])
        plt.show()
        plt.plot(2.0*np.pi*freq, 20.0*np.log10(2.0*T_fft))
        plt.gca().set_xlim([1.0-3.0*omega_m, 1.0+3.0*omega_m])
        plt.gca().set_ylim([-40.0, 0.0])
        plt.show()


max(e_max): 0.537440071914
max(dsig_max): 99999999.8126
(360, 107) (360, 107)


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib

# Set up formatting for the movie files
#Writer = animation.writers['mencoder']    ## avconv ffmpeg mencoder
#writer = Writer(fps=15, metadata=dict(artist='Me'))
#writer = animation.FFMpegWriter()

frame_start = int(7.0*n_saved/10)
n_frames = n_saved - frame_start


plt_field = 'Sig'
save_anim = True

logscale=False
log_0 = np.log(e_max)-5

use_subplots = False

plt.rcParams["figure.figsize"] = (5, 2)
font = {'family' : 'serif', 'weight' : 'normal', 'size'   : 14}
matplotlib.rc('font', **font)


fig = None
if n_dim>=2:
    def animate_E(i):
        plt.clf()
        #fig = plt.pcolor(y_E, z_E, E_out_list[i])
        if not logscale:
            #fig = plt.imshow(np.abs(E_out_list[i]).T, vmin=0.0, vmax=e_max/100)
            fig = plt.imshow(E_out_list[frame_start+i].T,  origin='lower', vmin=-e_max, vmax=e_max, aspect='auto')
        else:
            E_log = np.log(np.abs(E_out_list[frame_start+i]))
            E_log = (E_log>=log_0)*E_log + (E_log<log_0)*(log_0)
            fig = plt.imshow(E_log.T, origin='lower', vmin=log_0, vmax=np.log(e_max), aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        return fig

    def animate_Eps(i):
        plt.clf()
        fig = plt.imshow(deps_out_list[frame_start+i].T, origin='lower', vmin=-deps_max, vmax=deps_max, aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        return fig

    def animate_Sig(i):
        plt.clf()
        fig = plt.imshow(dsig_out_list[frame_start+i].T, origin='lower', vmin=-dsig_max, vmax=dsig_max, aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        return fig

    
    def animate_J(i):
        plt.clf()
        #fig = plt.pcolor(y_J, z_J, J_out_list[i])
        fig = plt.imshow(J_out_list[frame_start+i])
        axes = plt.gca()
        CB = plt.colorbar(fig, shrink=0.8, extend='both')
        return fig


    fig = plt.figure()
    axes = plt.gca()
    anim = None
    if plt_field=='E':
        anim = animation.FuncAnimation(fig, animate_E, frames=n_frames, interval=1, repeat=False)
    elif plt_field=='Eps':
        anim = animation.FuncAnimation(fig, animate_Eps, frames=n_frames, interval=1)
    elif plt_field=='Sig':
        anim = animation.FuncAnimation(fig, animate_Sig, frames=n_frames, interval=1)
    elif plt_field=='J':
        anim = animation.FuncAnimation(fig, animate_J, frames=n_frames, interval=1)
    else:
        raise ValuError()

    if save_anim:
        anim.save('other/e-2d.gif', writer="imagemagick", fps=15, dpi=200)
    plt.show()
elif n_dim==1:

    fig, axarr = None, None
    def animate_E(i):
        #global fig
        if not is_t_var:
            plt.clf()
            fig = plt.plot(x_E, E_out_list[i])
            axes = plt.gca()
            axes.set_ylim([-e_max, e_max])
            return fig
        else:
            if not use_subplots:
                plt.clf()
                fig = plt.plot(x_E, E_out_list[i])
                fig = plt.plot(x_deps, deps_out_list[i], 'r')
                #fig.set_tight_layout(True)
                axes = plt.gca()
                axes.set_ylim([-e_max, e_max])
                return fig
            else:
                plt.clf()
                fig, axarr = plt.subplots(2, sharex=True)
                axarr[0].plot(x_E, E_out_list[i])
                axarr[0].set_ylim([-e_max, e_max])
                axarr[1].plot(x_deps, deps_out_list[i], 'r')
                axarr[1].set_ylim([-deps_max, deps_max])
                return fig
            

    def animate_J(i):
        plt.clf()
        fig = plt.plot(x_J, J_out_list[i])
        axes = plt.gca()
        axes.set_ylim([-j_max, j_max])
        return fig

    anim = None
    if is_t_var and use_subplots:
        fig, axarr = plt.subplots(2, sharex=True)
    else:
        fig = plt.figure()

    if plt_field=='E':
        anim = animation.FuncAnimation(fig, animate_E, frames=n_saved, interval=1)
    elif plt_field=='J':
        anim = animation.FuncAnimation(fig, animate_J, frames=n_saved, interval=1)
    else:
        raise ValuError()

    if save_anim:
        anim.save('other/e-1d.gif', writer='imagemagick', fps=15)
    plt.show()
    
    

### surface plasmon bulk

In [4]:
import numpy as np
import matplotlib.pyplot as plt


def getEpsDrude(omega, sig_0, tau_0):
    sig = sig_0/(1-1j*omega*tau_0)
    eps = 1.0 - sig/(1j*omega)
    return eps

def getBulkPlasmonDispersion(omega, eps_1, eps_2):
    q = omega*np.sqrt(eps_1*eps_2/(eps_1+eps_2))
    q = q*(np.imag(q)<=0) + np.conjugate(q)*(np.imag(q)>0) 
    alpha_1 = np.sqrt(q**2 - omega**2*eps_1)
    alpha_2 = np.sqrt(q**2 - omega**2*eps_2)
    return [q, alpha_1, alpha_2]
    

def getPlasmonDispTopAir(omega, sig_0, tau_0):
    eps = getEpsDrude(omega, sig_0, tau_0)
    [q, alpha_1, alpha_2] = getBulkPlasmonDispersion(omega, eps_1=1, eps_2=eps)
    return [q, alpha_1, alpha_2]

sig_0 = 1.0e7
tau_0 = 1.0e6
omega = 2.222

eps = getEpsDrude(omega, sig_0, tau_0)
print('eps: ', eps)

q, alpha_1, alpha_2 = getBulkPlasmonDispersion(omega, eps_1=1, eps_2=eps)

print('q:', q, 'alpha_1:', alpha_1, 'alpha_2:', alpha_2, sep='\n')

n_pt = 1000
w = np.linspace(0, 2.5, n_pt)
eps = getEpsDrude(w, sig_0, tau_0)
q, alpha_1, alpha_2 = getBulkPlasmonDispersion(w, eps_1=1, eps_2=eps)
#print(q)

plt.plot(np.real(q), w)
plt.grid()
plt.show()


eps:  (-1.025405060757691+9.115234296839294e-07j)
q:
(14.1166505367-0.000246975463805j)
alpha_1:
(13.9406792652-0.000250093001018j)
alpha_2:
(14.2948430692-0.00024405420345j)


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in multiply


### Graphene PPWG dispersion

In [3]:
import numpy as np
import scipy.constants as constants

from Electromagnetics.Misc import solveMuller
from Electromagnetics.graphene import *

import matplotlib.pyplot as plt


def f_ppwg_disp_odd(beta, omega, d, sig):
    k_0 = omega/constants.c
    k_z = -1j*np.sqrt(beta**2 - k_0**2 + 0j)
    Y = omega*constants.epsilon_0/k_z
    Y_sig = sig
    f = (Y + Y_sig)*np.sin(k_z*d/2) - 1j*Y*np.cos(k_z*d/2)
    return f

def f_ppwg_disp_even(beta, omega, d, sig):
    k_0 = omega/constants.c
    k_z = -1j*np.sqrt(beta**2 - k_0**2 + 0j)
    Y = omega*constants.epsilon_0/k_z
    Y_sig = sig
    f = (Y + Y_sig)*np.cos(k_z*d/2) + 1j*Y*np.sin(k_z*d/2)
    return f


def get_ppwg_disp_odd(omega, d, sig, x_0):
    def f(beta):
        return f_ppwg_disp_odd(beta, omega, d, sig)
    
    x_1, x_2 = 1.1*x_0, 1.2*x_0
    res = solveMuller(f, x_0, x_1, x_2, toly_abs=1.0e-6, toly_rel=None)
    
    return res

def get_ppwg_disp_even(omega, d, sig, x_0):
    def f(beta):
        return f_ppwg_disp_even(beta, omega, d, sig)
    
    x_1, x_2 = 1.1*x_0, 1.2*x_0
    res = solveMuller(f, x_0, x_1, x_2, toly_abs=1.0e-6, toly_rel=None)
    
    return res

def get_ppwg_disp_graph_odd(omega, mu_c, tau, d, x_0):
    sigma_d, sigma_o = condKuboLorentzian(mu_c=mu_c, B_0=0, tau=tau, omega=np.array([omega]), T=300) 
    sig = sigma_d[0]
    return get_ppwg_disp_odd(omega, d, sig, x_0)

def get_ppwg_disp_graph_even(omega, mu_c, tau, d, x_0):
    sigma_d, sigma_o = condKuboLorentzian(mu_c=mu_c, B_0=0, tau=tau, omega=np.array([omega]), T=300) 
    sig = sigma_d[0]
    return get_ppwg_disp_even(omega, d, sig, x_0)

def get_ppwg_qtem_graph(omega, mu_c, tau, d):
    k_0 = omega/constants.c
    eta_0 = np.sqrt(constants.mu_0/constants.epsilon_0)
    sigma_d, sigma_o = condKuboLorentzian(mu_c=mu_c, B_0=0, tau=tau, omega=np.array([omega]), T=300) 
    sig = sigma_d[0]
    beta_k0 = np.sqrt(1.0 - 1j*2.0/(sig*eta_0*k_0*d))
    return beta_k0*k_0


mu_c = 0.3*constants.eV
tau = 0.2*constants.pico

omega = 1.0*constants.tera
d = 20.1*constants.micro
sig = 1.0*constants.milli

res =  get_ppwg_disp_graph_odd(omega, mu_c, tau, d, 1.0e6)

print(res)


n_pt = 20
omega = np.linspace(0.01, 3.0, n_pt)*constants.tera

"""
beta_odd = np.zeros(n_pt)
for i in range(n_pt):
    x_0 = beta_odd[i-1]
    if i==0:
        x_0 = 1.0e6
    res =  get_ppwg_disp_graph_odd(omega[i], mu_c, tau, d, x_0)
    assert res[3]==None
    beta_odd[i] = res[0]
"""
    
beta_odd_2 = np.zeros(n_pt)
for i in range(n_pt):
    beta_odd_2[i] = get_ppwg_qtem_graph(omega[i], mu_c, tau, d)

                        
"""
beta_even = np.zeros(n_pt)
for i in range(n_pt):
    x_0 = beta_even[i-1]
    if i==0:
        x_0 = 1.0
    res =  get_ppwg_disp_graph_even(omega[i], mu_c, tau, d, x_0)
    if res[3]!=None:
        print(res)
        assert False
    beta_even[i] = res[0]
"""

mu_c *= 2.0
B_0 = 0.0
k_k0_p, k_k0_m = magnetoplasmonDispersion_normalized(mu_c, B_0, omega, tau, T=300, cond='Kubo')

    
#plt.plot(np.real(beta_odd), omega, 'b')
#plt.plot(-np.real(beta_odd), omega, 'b')

plt.plot(np.real(beta_odd_2)/1.0e4, omega, 'b')
plt.plot(-np.real(beta_odd_2)/1.0e4, omega, 'b')

#plt.plot(np.real(beta_even), omega)
#plt.plot(omega, np.real(beta_odd), 'b')
plt.plot(np.real(k_k0_p)*omega/constants.c/1.0e4, omega, 'r')
plt.plot(-np.real(k_k0_p)*omega/constants.c/1.0e4, omega, 'r')
plt.show()

[(-133.18952952427586+937756.26588416519j), (-2.0633362822750776e-13-2.436721968475321e-12j), 24, None]


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:92: ComplexWarning: Casting complex values to real discards the imaginary part
